In [10]:
# Retrieving Waste Bank Data via Google Apps Script API

# import library requests and json
import requests
import json

# API key for accessing the Google Apps Script API
apiKey = 'a8b3e6d4eaee404ba7b32cd80110a2eb' 
url = 'https://script.google.com/a/upi.edu/macros/s/AKfycbwqznIpOsEC_cPAe4pTy4pEQBN5MaksSaIy0IL8kganMDDeDYchgsebQb1yeEoSpxet/exec?api=sampah&apiKey='

# Construct the URL with the API key
url += apiKey  

response = requests.get(url)  # Send a GET request to the URL
data = response.json()  # Parse the response as JSON

print(json.dumps(data, indent=4))  # Print the JSON data with indentation

{
    "data": [
        {
            "date": "17-01-2024",
            "name": "Masood Ghani",
            "wasteType": "Dry Box",
            "weight": 5.1,
            "price": 4079.9999999999995
        },
        {
            "date": "17-01-2024",
            "name": "Masood Ghani",
            "wasteType": "Mixed Bucket",
            "weight": 2,
            "price": 1000
        },
        {
            "date": "17-01-2024",
            "name": "Masood Ghani",
            "wasteType": "Aqua Clean Bottle",
            "weight": 0.8,
            "price": 1600
        },
        {
            "date": "17-01-2024",
            "name": "Masood Ghani",
            "wasteType": "Crackle",
            "weight": 1.6,
            "price": 640
        },
        {
            "date": "17-01-2024",
            "name": "Masood Ghani",
            "wasteType": "Dry Duplex",
            "weight": 1,
            "price": 400
        },
        {
            "date": "17-01-2024",
            "n

In [11]:
# Displaying Waste Bank Data in a DataFrame
import pandas as pd

df = pd.DataFrame(data['data'])

display(df)

,date,name,wasteType,weight,price
0,17-01-2024,Masood Ghani,Dry Box,5.1,4080.0
1,17-01-2024,Masood Ghani,Mixed Bucket,2.0,1000.0
2,17-01-2024,Masood Ghani,Aqua Clean Bottle,0.8,1600.0
3,17-01-2024,Masood Ghani,Crackle,1.6,640.0
4,17-01-2024,Masood Ghani,Dry Duplex,1.0,400.0
...,...,...,...,...,...
461,12-06-2024,Muhammad Abu Aqeel,Mixed Bucket,0.6,300.0
462,12-06-2024,Muhammad Abu Aqeel,Mixed Bucket,0.5,250.0
463,12-06-2024,Muhammad Abu Aqeel,LD/gallon cap,0.1,200.0
464,12-06-2024,Muhammad Abu Aqeel,Mixed Bucket,0.1,50.0


In [17]:
import plotly.express as px  # Importing the plotly express module for creating interactive plots
import plotly.graph_objs as go  # Importing the plotly graph objects module for creating customized plots

# Total Revenue and Total Waste
# Scorecard Text
total_revenue = df['price'].sum()  # Calculating the total revenue by summing the 'price' column
total_waste = df['weight'].sum()  # Calculating the total waste by summing the 'weight' column

# Top 5 Waste Type
# Horizontal Bar Chart
wasteType_sum = df.groupby('wasteType')['weight'].count().reset_index()  # Grouping the dataframe by 'wasteType' and counting the occurrences of each type
wasteType_sum_sort = wasteType_sum.sort_values(by='weight', ascending=False)  # Sorting the waste types by their count in descending order
wasteType_sum = wasteType_sum_sort.sort_values(by='weight', ascending=False).head(5)  # Selecting the top 5 waste types
fig_wasteType = px.bar(wasteType_sum, x='weight', y='wasteType', orientation='h', title='Type of Waste')  # Creating a horizontal bar chart

# 4 Newest Date
# Line Chart
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')  # Converting the 'date' column to datetime format
waste_trend = df.groupby('date')['weight'].count().reset_index()  # Grouping the dataframe by 'date' and counting the occurrences of each date
newest_waste_trend = waste_trend.sort_values(by='date', ascending=False)  # Sorting the waste trend by date in descending order
newest_waste_trend = newest_waste_trend.sort_values(by='date', ascending=False)  # Sorting the waste trend by date in descending order (repeated line)
fig_waste_trend = px.line(newest_waste_trend, x='date', y='weight', orientation='h', title='Waste Trend')  # Creating a line chart

# Most Consistent Customers
# Heat Map
# Calculate the number of unique dates per customer
consistent_customers = df.groupby('name')['date'].nunique().reset_index()
consistent_customers.columns = ['name', 'unique_dates']  # Rename columns for clarity
# consistent_customers = df['name'].value_counts().reset_index()  # Counting the occurrences of each customer name
fig_consistent_customers = px.treemap(consistent_customers, path=[consistent_customers.name], values='unique_dates', title='Most Consistent Customers')  # Creating a treemap

# Target Customer
# Gauge Chart
target_customer_count = df['name'].nunique()  # Counting the number of unique customer names
max_target_customer = 50  # Set the maximum target customer count here
fig_target_customer = go.Figure(go.Indicator(
    mode="gauge+number",
    value=target_customer_count,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Target Customer"},
    gauge={'axis': {'range': [None, max_target_customer]},
           'bar': {'color': "darkblue"},
           'steps': [
               {'range': [0, max_target_customer / 2], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': target_customer_count}}))
fig_target_customer.update_layout()

# Customer vs Total Revenue
# Scatter Plot
fig_scatter_plot = px.scatter(df, x='price', y='name', title='Customer vs Total Revenue')  # Creating a scatter plot

In [18]:
# Install the required libraries
!pip install dash jupyter_dash

# Import the required libraries
from dash import Dash, dcc, html
from jupyter_dash import JupyterDash

# Create the Dash app layout
app = JupyterDash(__name__, external_stylesheets=['https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css'])
app.layout = html.Div([
    html.Div([
        html.H1('Waste Bank Dashboard', className='text-center my-5'),
        html.Div([
            html.Div([
                html.Div([
                    html.P('Total Revenue', className='h3'),
                    html.P(f'Rp {total_revenue}', className='h1'),
                ]),
                html.Div([
                    html.P('Total Waste', className='h3'),
                    html.P(f'{total_waste} kg', className='h1'),
                ], className='mt-5'),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_wasteType),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_waste_trend),
            ], className='col-lg-4 card'),
        ], className='row align-items-center'),
        html.Div([
            html.Div([
                dcc.Graph(figure=fig_consistent_customers),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_target_customer),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_scatter_plot),
            ], className='col-lg-4 card'),
        ], className='row align-items-center'),
    ], className="container")
])

# Run the app
if __name__ == '__main__':
  app.run()


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

